In [12]:
import pandas as pd
import numpy as np
import sys
import re
from progressbar import ProgressBar
#import multiprocessing as mp

In [13]:
def core_tcrf(core):
    core_tcrf = f'___{core[3]}{core[4]}{core[5]}{core[6]}{core[7]}_'
    return core_tcrf

In [14]:
## non-cross conserved  class I eptiopes determined by TCRf 4,5,6,7,8
def epitope_distance(core1,core2,score1,score2):
    if core1[3]==core2[3] and core1[4]==core2[4] and core1[5]==core2[5] and core1[6]==core2[6] and core1[7]==core2[7]:
        return 0
    else:
        return score1+score2

In [15]:
## example to calculate sequence distance with seq[1] and seq[2]
## i and are j are the index of seq list

def seq_dist(df_eptiope,i,j):
    seq = df_epitope.ID.unique()
    seq1_epitope = df_eptiope.loc[df_eptiope['ID']==seq[i]]
    seq2_epitope = df_eptiope.loc[df_eptiope['ID']==seq[j]]
    epitope_compare = pd.merge(seq1_epitope, seq2_epitope, how="outer", on=["Pos","core_tcrf"])
    epitope_compare['EL-score_x']=epitope_compare['EL-score_x'].fillna(0)
    epitope_compare['EL-score_y']=epitope_compare['EL-score_y'].fillna(0)
    epitope_compare['core_x']=epitope_compare['core_x'].fillna('---------')
    epitope_compare['core_y']=epitope_compare['core_y'].fillna('---------')
    epitope_compare['epi_dist'] = epitope_compare.apply(lambda x: epitope_distance(str(x['core_x']),str(x['core_y']),x['EL-score_x'],x['EL-score_y']), axis =1)
    seq_distance = epitope_compare['epi_dist'].sum()
    return(seq_distance)


In [22]:
## loop through all seqs in the eitope prediction dataframe to get the no-conserved epitope distance

def non_conserved_d(df_epitope):
    pbar = ProgressBar()
    seq = df_epitope.ID.unique()
    seq_distance=[]
    for i in pbar(range(len(seq))):
        for j in range(len(seq)):
            if i<j:
                distance = seq_dist(df_epitope,i,j)
                seq_distance.append([seq[i], seq[j], distance])                
                df_distance = pd.DataFrame(seq_distance, columns=["seq1", "seq2","epitope_distance"])           
    return(df_distance)


In [23]:
def get_allele(allel_xls):
    with open(allel_xls, "r") as file:
        first_line = file.readline()
        allele=re.sub(r"\t|\n", "", first_line)
    return allele    

In [24]:
## input is the netMPHpan xlsx for multiple sequence but single allel
## for single allel, load the xls file and filter by cutoff
netMHCpan_out="netMHCpan_out/HLAA0101.xls"
data = pd.read_csv(netMHCpan_out,sep="\t",skiprows=1)
df_epitope = data.loc[data['EL_Rank'] <2] 
df_epitope['core_tcrf'] = df_epitope.apply(lambda x:core_tcrf(x['core']),axis=1)
df_epitope

/var/folders/5l/1xy2811j5n1dzz9s802679j80000gr/T/ipykernel_73872/3765877528.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_epitope['core_tcrf'] = df_epitope.apply(lambda x:core_tcrf(x['core']),axis=1)


,Pos,Peptide,ID,core,icore,EL-score,EL_Rank,Ave,NB,core_tcrf
22,22,ASSQNITEEFY,KF826828-A-2004,ASSQTEEFY,ASSQNITEEFY,0.5147,0.2796,0.5147,1,___QTEEF_
33,33,QSTCSAVSKGY,KF826828-A-2004,QSSAVSKGY,QSTCSAVSKGY,0.0916,1.3607,0.0916,1,___AVSKG_
187,187,LTSKVLDLKNY,KF826828-A-2004,LTSKVLDLY,LTSKVLDLKNY,0.3065,0.5307,0.3065,1,___KVLDL_
239,239,NAGVTTPVSTY,KF826828-A-2004,NATTPVSTY,NAGVTTPVSTY,0.0786,1.5013,0.0786,1,___TPVST_
288,288,MSIIKEEVLAY,KF826828-A-2004,MSIEEVLAY,MSIIKEEVLAY,0.2965,0.5475,0.2965,1,___EEVLA_
...,...,...,...,...,...,...,...,...,...,...
920309,242,VTTPVSTY,RSVA_pda_ancest,VTTPV-STY,VTTPVSTY,0.2795,0.5762,0.2795,1,___PV-ST_
920365,298,YVVQLPLY,RSVA_pda_ancest,YVVQLP-LY,YVVQLPLY,0.0541,1.8887,0.0541,1,___QLP-L_
920450,383,VDIFNPKY,RSVA_pda_ancest,-VDIFNPKY,VDIFNPKY,0.1329,1.0487,0.1329,1,___IFNPK_
920516,449,VSVGNTLY,RSVA_pda_ancest,VSV-GNTLY,VSVGNTLY,0.1002,1.2851,0.1002,1,___-GNTL_


In [25]:
seq_epidist=non_conserved_d(df_epitope)
allele=get_allele(netMHCpan_out)
seq_epidist['allele']=allele
seq_epidist


100% |#############################################################################################################################################################################################################################|


,seq1,seq2,epitope_distance,allele
0,KF826828-A-2004,KF826827-A-2004,0.0000,HLA-A01:01
1,KF826828-A-2004,KF530260-A-2005,0.0670,HLA-A01:01
2,KF826828-A-2004,KF826838-A-2006,0.9483,HLA-A01:01
3,KF826828-A-2004,KF826841-A-2007,0.0000,HLA-A01:01
4,KF826828-A-2004,KF826846-A-2008,0.0000,HLA-A01:01
...,...,...,...,...
82616,AF035006_vaccin,RSVB_pda_ancest,17.5691,HLA-A01:01
82617,AF035006_vaccin,RSVA_pda_ancest,1.8555,HLA-A01:01
82618,KT992094_vaccin,RSVB_pda_ancest,17.5691,HLA-A01:01
82619,KT992094_vaccin,RSVA_pda_ancest,1.8555,HLA-A01:01


In [26]:
seq_epidist.to_csv("netMHCpan_out/HLAA0101_seqdist.csv",index=False)

In [ ]:
## start Parallelizing using Pool.apply() https://www.machinelearningplus.com/python/parallel-processing-python/
results = []
for row in data:
    results.append(howmany_within_range(row, minimum=4, maximum=8))
    
results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]


def non_conserved_d(df_epitope):
    pbar = ProgressBar()
    seq = df_epitope.ID.unique()
    seq_distance=[]
    for i in pbar(range(len(seq))):
        for j in range(len(seq)):
            if i<j:
                distance = seq_dist(df_epitope,i,j)
                seq_distance.append([seq[i], seq[j], distance])
                
                seq_distance=[pool.apply(seq[i],seq[j],seq_dist,args=(df_epitope,i,j) for j in range (len(seq)) and i < j]
                df_distance = pd.DataFrame(seq_distance, columns=["seq1", "seq2","epitope_distance"])           
    return(df_distance)


In [27]:
## problem with this approach: 
## more sesquence, more dimentions, time consuming